In [1]:
import os
import random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import time

import sys 

sys.path.insert(1, 'D:/IPS_PCA_CLUSTERING-the_end/')

import torch
from torch.utils.data import DataLoader

from copy import deepcopy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression, SGDClassifier

# custom
from src.utils.dataset import RSSIDataset
from src.utils.file_utils import read_data, load_data, load_csv
from src.utils.helper import get_current_date, normalize_data, separates_data_uts, mean_error, clip_gradient, \
    constant_columns, detect_num_feature_rssi, load_data_npz, draw_distribution_cluster, draw_kmean_distortion, \
    draw_kmean_silhouette, set_seed_everything
from src.utils.models import RegressionNet, ClassificationNet, Ensemble_Model, Ensemble_Classic_Model, train_regression, \
    train_classification, model_evaluation, classic_model_evaluation, load_model_classification, save_model_sklearn, \
    load_model_sklearn
from src.utils.loss import MeanLoss
import src.utils.regr_utils as regr

SEED = 2023
set_seed_everything(SEED)

### Define model path

In [2]:
date_str = get_current_date()
# date_str = "2023_02_05"
n_cluster_str = 2
idx_data_type = 0
data_type_str = ["uts", "uji", "tampere"]
detail_data = "kernal_pca_cosine_250"

all_path = {
    'type': 'kernel_pca_250 | Tampere | 2_cluster_label',
    'n_clusters': 2,
    'result_path': f'../../results/floor_classification/{data_type_str[idx_data_type]}/{detail_data}.csv',
    'data_dir': f"../../data/{data_type_str[idx_data_type]}_data/pca/{detail_data}/",
    'cluster': f'../../models/{data_type_str[idx_data_type]}_models/cluster_{n_cluster_str}/{date_str}/model_cluster.sav',
    'cluster_classifier': f'../../models/{data_type_str[idx_data_type]}_models/cluster_{n_cluster_str}/{date_str}/model_cluster_classifier.sav',
    'ensemble_classifier': f'../../models/{data_type_str[idx_data_type]}_models/cluster_{n_cluster_str}/{date_str}/ensemble_classifier',
    'floor_classifier': f'../../models/{data_type_str[idx_data_type]}_models/cluster_{n_cluster_str}/{date_str}/model_floor_classifier.sav',
}
Path(all_path["ensemble_classifier"]).mkdir(parents=True, exist_ok=True)
Path(f'../../models/{data_type_str[idx_data_type]}_models/cluster_{n_cluster_str}/{date_str}/').mkdir(parents=True, exist_ok=True)
data_type_str[idx_data_type]

'uts'

### Load Data

In [3]:
#data_dir = "../../data/uts_data/pca/kernal_pca_cosine_100/"
data_dir = all_path["data_dir"]

(X_train, Y_train), (X_test, Y_test), (X_valid, Y_valid) = load_data_npz(
    [
        os.path.join(data_dir, "X_train.npz"),
        os.path.join(data_dir, "Y_train.npz"),
        os.path.join(data_dir, "X_test.npz"),
        os.path.join(data_dir, "Y_test.npz"),
        os.path.join(data_dir, "X_valid.npz"),
        os.path.join(data_dir, "Y_valid.npz")
    ]
)

# cluster_model = load_model_sklearn(all_path["cluster"])
# Z_train = cluster_model.predict(Y_train[:,:2])
# Z_test = cluster_model.predict(Y_test[:,:2])
# Z_valid = cluster_model.predict(Y_valid[:,:2])

X_train = np.vstack([X_train, X_valid])
Y_train = np.vstack([Y_train, Y_valid])

Y_train = Y_train[:, 2]
Y_test = Y_test[:, 2]
Y_valid = Y_valid[:, 2]

In [4]:
import os
import src.utils.cls_utils as clf
from src.utils.helper import get_result_floor_classifier

data_type = ["tampere", "uji", "uts"]

for data_prefix in data_type:
    for sub_folder in os.listdir(f"../../data/{data_prefix}_data/pca/"):
        # if sub_folder == "no_pca":
        #     continue
        print(f"************** {data_prefix} | {sub_folder} ************** ")
        data_dir = f"../../data/{data_prefix}_data/pca/{sub_folder}/"

        (X_train, Y_train), (X_test, Y_test), (X_valid, Y_valid) = load_data_npz(
            [
                os.path.join(data_dir, "X_train.npz"),
                os.path.join(data_dir, "Y_train.npz"),
                os.path.join(data_dir, "X_test.npz"),
                os.path.join(data_dir, "Y_test.npz"),
                os.path.join(data_dir, "X_valid.npz"),
                os.path.join(data_dir, "Y_valid.npz")
            ]
        )
        X_train = np.vstack([X_train, X_valid])
        Y_train = np.vstack([Y_train, Y_valid])

        Y_train = Y_train[:, 2]
        Y_test = Y_test[:, 2]

        get_result_floor_classifier(
            clf.models_trial,
            X_train, Y_train,
            X_test, Y_test,
            data_prefix,
            f"../../results/floor_classification/{data_prefix}/{sub_folder}.csv"
        )

    print("")

************** tampere | kernal_pca_cosine_75 ************** 
KNeighborsClassifier()
	Score: 0.8663629460895975 | Training Time: 0.11192536354064941
LogisticRegression()


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\IPS_PCA_CLUSTERING-the_end\src\utils\helper.py:228: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log = log.append(log_entry, ignore_index=True)
D:\IPS_PCA_CLUSTERING-the_end\src\utils\helper.py:228: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log = log.append(log_entry, 

	Score: 0.9334345735256897 | Training Time: 0.030046939849853516
SGDClassifier()
	Score: 0.9336876740065806 | Training Time: 0.010107755661010742
DecisionTreeClassifier(max_depth=5, random_state=2023)
	Score: 0.7727157681599595 | Training Time: 0.016002178192138672
RandomForestClassifier(max_depth=5, random_state=2023)
	Score: 0.8790179701341433 | Training Time: 0.1901721954345703
SVC(C=0.025, kernel='linear', random_state=2023)
	Score: 0.3844596304732979 | Training Time: 0.06834006309509277
SVC(C=1, gamma=2, random_state=2023)


D:\IPS_PCA_CLUSTERING-the_end\src\utils\helper.py:228: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log = log.append(log_entry, ignore_index=True)
D:\IPS_PCA_CLUSTERING-the_end\src\utils\helper.py:228: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log = log.append(log_entry, ignore_index=True)


	Score: 0.9291318653505442 | Training Time: 0.1441340446472168
SVC(probability=True, random_state=2023)
	Score: 0.9334345735256897 | Training Time: 0.19291186332702637
MLPClassifier(random_state=2023)


...

D:\IPS_PCA_CLUSTERING-the_end\src\utils\helper.py:228: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log = log.append(log_entry, ignore_index=True)


	Score: 0.8402061855670103 | Training Time: 166.59912657737732
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)


D:\IPS_PCA_CLUSTERING-the_end\src\utils\helper.py:228: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log = log.append(log_entry, ignore_index=True)


	Score: 0.9355670103092784 | Training Time: 6.9796435832977295



D:\IPS_PCA_CLUSTERING-the_end\src\utils\helper.py:228: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log = log.append(log_entry, ignore_index=True)


In [5]:
get_result_floor_classifier(model_list, X_train, Y_train, X_test, Y_test,
                            data_type=data_type_str[idx_data_type],
                            result_path=all_path["result_path"])

NameError: name 'model_list' is not defined

### Train floor Classifier

In [ ]:
# X_train_ = X_train[Z_train == 0]
# Y_train_ = Y_train[Z_train == 0]
#
# X_valid_ = X_valid[Z_valid == 0]
# Y_valid_ = Y_valid[Z_valid == 0]

In [ ]:
# m = LogisticRegression()
# m.fit(X_train_, Y_train_)
# m.score(X_valid_, Y_valid_)

In [ ]:
# import src.utils.cls_utils as clf
# en_model = clf.Ensemble_Classic_Model(
#     models= [
#         SGDClassifier(),
#         MLPClassifier(), LogisticRegression(),
#         SVC(kernel='rbf', probability=True),
#         XGBClassifier(), RandomForestClassifier(),
#         KNeighborsClassifier()
#     ],
#     ensemble_model_path=all_path["ensemble_classifier"]
# )
# en_model.train_student(X_train, Y_train, Z_train, X_valid, Y_valid, Z_valid)

In [ ]:
# clf.classic_model_evaluation(
#     model_dict_path=[
#         os.path.join(all_path["ensemble_classifier"], "model_0.sav"),
#         os.path.join(all_path["ensemble_classifier"], "model_1.sav"),
#         # os.path.join(all_path["ensemble_classifier"], "model_2.sav"),
#     ],
#     model_cluster_classifier_path=all_path["cluster_classifier"],
#     X_test=X_test,
#     Y_test=Y_test
# )

### train base 2

In [ ]:
# clf1 = MLPClassifier()
# clf2 = LogisticRegression()
# clf3 = SVC(kernel='rbf', probability=True)
# clf4 = XGBClassifier()
# clf5 = GaussianNB()
# clf6 = SGDClassifier()
# clf7 = KNeighborsClassifier()
# clf8 = DecisionTreeClassifier()
# clf9 = RandomForestClassifier()
# clf10 = GradientBoostingClassifier()
#
# eclf = VotingClassifier(
#     estimators=[
#         ('clf1', clf1), ('clf2', clf2), ('clf3', clf3)
#     ],
#     voting="soft",
#     weights=[3, 1, 2]
# )
#
# model_list = [clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf8, clf9, clf10, eclf]
#
# print(f"Train clf1 model: {clf1.__class__.__name__}")
# clf1 = clf1.fit(X_train, Y_train)
# print(clf1.score(X_test, Y_test))
#
# print(f"Train clf2 model: {clf2.__class__.__name__}")
# clf2 = clf2.fit(X_train, Y_train)
# print(clf2.score(X_test, Y_test))
#
# print(f"Train clf3 model: {clf3.__class__.__name__}")
# clf3 = clf3.fit(X_train, Y_train)
# print(clf3.score(X_test, Y_test))
#
# print(f"Train clf3 model: {clf4.__class__.__name__}")
# clf4 = clf4.fit(X_train, Y_train)
# print(clf4.score(X_test, Y_test))
#
# print(f"Train eclf model: {eclf.__class__.__name__}")
# eclf = eclf.fit(X_train, Y_train)
# print(eclf.score(X_test, Y_test))
#
# save_model_sklearn(all_path["floor_classifier"], eclf)

In [ ]:
# loaded_model = load_model_sklearn(all_path["floor_classifier"])
# result = loaded_model.score(X_test, Y_test)
# print("\nFinal")
# print(result)

arr_0
[[-0.32710986  0.26471843  0.2284243  ... -0.01962228  0.01599412
  -0.02166901]
 [-0.0887591   0.00370714 -0.01304901 ... -0.00221932 -0.00737536
  -0.00360871]
 [ 0.09886772  0.32281415 -0.1254653  ... -0.01247815 -0.00067846
  -0.02028829]
 ...
 [-0.30898421  0.11282028  0.29537248 ...  0.00162467 -0.03156896
   0.02052481]
 [ 0.1238168  -0.14711911 -0.20325805 ...  0.02832593 -0.00089344
  -0.02201901]
 [-0.11102505 -0.13316179 -0.15960501 ... -0.03597616  0.01809136
  -0.00819005]]
